<a href="https://colab.research.google.com/github/VL98/VLehner-Bachelor-Thesis/blob/main/EGP_Run2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EPG Calibration

### Copyright Vinzenz Lehner (University of St. Gallen)

### Install packages

In [ ]:
#@title
install.packages("ragtop")
install.packages("pracma")
install.packages("MASS")
install.packages("nloptr")
install.packages("NlcOptim")
install.packages("rootSolve")
install.packages("stats")
install.packages("parallel")
install.packages("timeDate")
install.packages("timeSeries")

### Loading libraries and functions

In [ ]:
#@title
library(ragtop)
library(pracma)
library(MASS)
library(nloptr)
library(NlcOptim)
library(rootSolve)
library(stats)
library(parallel)
library(timeDate)
library(timeSeries)
library(googledrive)
library(futile.logger)
library(limSolve)

# library(foreach)
# library(doParallel)
# library(iterators)


seed = set.seed(1004)
# --------------------------------------------------


# option <- read.csv("option15clean.csv")
# option <- read.csv("option10clean.csv")

# v = unique(option$date)
n = 10^5
# x_0 = read.csv("EGP_2010_Day1.csv")$x
# x_2 = as.numeric(x_2$x[2:12])

# seed = set.seed(1004)
# gnoise = rnorm(n*151) 
# sa = sample(1:(n*151))

# --------------------------------------------------

# DayNo = 30
# date = v[DayNo]

# n = 100000                ######### parameters of the model
# t_size = option$date == date
# date = date

# rate = function(z){
#   option[ option$date == date & option$TTM == z,  ]$interest[1]/(100*252)
# }
# d = unique(option$TTM[t_size])
# d = unique(option$TTM)
# r0 = median(sapply(d, rate))

# ModelName = paste("EPG_Model_Day", DayNo, "_", date,".Rdata",sep = "")

# --------------------------------------------------


scaler = function(x){
  x/(10^ceil(log10(abs(x_0))))
}

rescaler = function(x){
  x*(10^ceil(log10(abs(x_0))))
}

hin3 = function(x) {
  x = rescaler(x)
  h = numeric(1)
  h[1] = x[2]*(x[4]+ 2)^2  + x[3] - 1 + 1e-15
  return(h)
}

heq3 = function(x) {
  x = rescaler(x)
  h = numeric(2)
  h[1] = x[10]*x[6] + (1-x[10])*x[7]
  h[2] =  x[10]*(x[6]^2)  + (1-x[10])*(x[7]^2)  + x[10]*(x[8]^2) + (1-x[10])*(x[9]^2) - 1
  return(h)
}

# --------------------------------------------------


fN2 = function(x){
  x = rescaler(x)
  # n = 100000
  set.seed(400)
  # t_size = option$date == date
  # date = date
  om = x[1]
  a = x[2]
  b = x[3]
  g = x[4]
  l = 0
  h_0 = x[5]
  mu_1 = x[6]
  mu_2 = x[7]
  sd_1 = x[8]
  sd_2 = x[9]
  p = x[10]
  E = p*mu_1  +  (1-p)*mu_2
  
  H = p*(mu_1^2)  + (1-p)*(mu_2^2)  + p*(sd_1^2) + (1-p)*(sd_2^2) - E^2
  
  ## M_1 = sqrt(1/H)*mu_1 - sqrt(1/H)*E
  M_1 = mu_1                                  ###### real mu_1: there is a second option to incoroporate the constraint
  ##  M_2 = sqrt(1/H)*mu_2 - sqrt(1/H)*E
  M_2 = mu_2
  ## S_1 = sd_1/sqrt(H)
  S_1 = sd_1                              #### same with the variance
  ## S_2 = sd_2/sqrt(H)
  S_2 = sd_2    
  # rate = function(z){option[ option$date == date & option$TTM == z,  ]$interest[1]/(100*252)}
  

  # d = unique(option$TTM[t_size])

  
  G = function(u , v,  mean = F){
    
    m = (0 + v) * mean
    y = p*exp((m + sqrt(v)*M_1)*u  + (u^2)*(S_1^2)*v/2) +
      (1-p)*exp((m + sqrt(v)*M_2)*u  + (u^2)*(S_2^2)*v/2) 
    return(y)
  }
  
  #### either trading days or regular days
  
  psize = 1:round(p*n*151)
  noise = c((M_1 + (sd_1)*gnoise[psize]),( M_2 + (sd_2)*gnoise[-psize]))[sa]
  e = matrix(data = noise[1:(max(d)*n)], nrow = n, ncol = max(d))
  
  h =  matrix(data = 0, nrow = n, ncol = max(d))
  
  h[ ,1] = h_0
  S = matrix(data = 0, nrow = n, ncol = max(d))
  S[ ,1] = exp(sqrt(h[ ,1])*e[ ,1] - log(G(1, h[ ,1], F)))
  
  for(t in 2:(max(d))){   ### carefull with the e
  # foreach(t=2:max(d), .combine=cbind) %dopar% {
    
    h[ , t] = om + a*(e[ ,t-1] - sqrt(h[ , t -1])*(l + g) - log(G(1, h[ , t-1], F))/sqrt(h[ , t -1]) )^2  + b*h[ ,t-1]
    S[ , t] = S[ , t-1] * exp(sqrt(h[ ,t])*e[ ,t] - log(G(1, h[ ,t], F))) 
    S[ , t] =  S[ , t]/mean(S[ , t])
  }
  
  ##y = option[t_size, c(3, 14, 18)]
  ###3
  A = function(y){ #######!!!!!!!! be careful with the indexes in the data set
    y = as.numeric(y)
    T_m = y[3]
    S_0 = y[2] ### or futures price
    # r = rate(T_m)
    r = ratex[ d == T_m]
    R = S[ , T_m] * S_0*exp(r*T_m) 
    K = y[1]
    C = mean((R > K)*(R - K)*exp(-r*T_m))
    IV = try( implied_volatilities(C, 1, S_0, K, r*252, T_m/252))
    ifelse(class(IV) == "try-error"| is.na(IV), 4, IV)
    # C
  }
  
  # CP = apply(option[t_size, c("strike_price", "SPOT", "TTM") ],1, A)
  ##mean((CP - option$price[t_size])^2)
  # mean((CP - option[t_size, "own_impvol"])^2)

  prices.list <- split(option[t_size, c("strike_price", "SPOT", "TTM")], seq(nrow(option[t_size, c("strike_price", "SPOT", "TTM")])))
  CP = mclapply(prices.list, A, mc.cores = mccores)
  mean((unlist(CP) - option[t_size, "own_impvol"])^2)
  
}

# --------------------------------------------------

FN2_C_Final = function(x){
  a = try(fN2(x))
  ifelse(class(a)== "try-error"| is.na(a), 16, a)
}
# --------------------------------------------------

# boundslow = c( 1e-12, 0, 0, -1500, 1e-12, -15, -15, 1e-12, 1e-12, 0, -20 ) 
# boundsup = c( 1, 1, 1, 1500, 1, 15, 15, 10, 20, 1, 20)

#

Loading required package: limSolve

Loading required package: futile.logger

Welcome to ragtop.  Logging can be enabled with commands such as
  futile.logger::flog.threshold(futile.logger::INFO, name='ragtop.calibration')


Attaching package: ‘rootSolve’


The following objects are masked from ‘package:pracma’:

    gradient, hessian




### Defining global variables

In [ ]:
#@title
# option <- read.csv("option15clean.csv")
mccores = 4

year=19
option_File = paste("option",year,"clean.csv", sep = "")
option <- read.csv(option_File)

v = unique(option$date)
# n = 10^5
n=100000

x_0 = read.csv("EGP_2019_Day1_x_0.csv")$x
# x_2 = as.numeric(x_2$x[2:12])

seed = set.seed(1004)
gnoise = rnorm(n*151) 
sa = sample(1:(n*151))

# system.time(fN2(scaler(x_0)))
# fN2(scaler(x_0))


boundslow = c(1e-12, 0, 0, -1500, 1e-12, -15,  -15, 1e-12, 1e-12, 0) 
boundsup  = c(0.6, 0.6, 1, 1500, 1, 15, 15, 10, 20, 1)


### Executing calibration calculation

In [ ]:
#@title

for (DayNo in c(15, 20)) {
  # DayNo = 30
  date = v[DayNo]

  # n = 100000                ######### parameters of the model
  t_size = option$date == date

  d = unique(option$TTM[t_size])
  rate = function(z){
    option[ option$date == date & option$TTM == z,  ]$interest[1]/(100*252)
  }
  ratex =  sapply(d, rate)
  # d = unique(option$TTM)
  r0 = median(ratex)
  # print(ratex)
  
  spotprice = option[ option$date == date,  ]$SPOT[1]
  EGPModelName = paste("EGP_Model_20",year,"_Day", DayNo, "_", date,".Rdata",sep = "")
  print(EGPModelName)

  start = Sys.time()
  EGP_Model = fmincon(scaler(x_0), FN2_C_Final, lb = scaler(boundslow), ub = scaler(boundsup), hin =  hin3, heq = heq3, tol = 1e-06 )

  EGP_resacle = rescaler(EGP_Model$par)

  dur = Sys.time() - start
  print(dur) 
  # Model$par

  save(EGP_Model, file = EGPModelName)
}

# mccores = 2
# system.time(fN2(scaler(x_0)))
# fN2(scaler(x_0))
# scaler(Model$par)
